In [1]:
import numpy as np
import pandas as pd

In [80]:
df = pd.read_csv("../Resources/allzipcodes_collision_weather_df.csv")

/var/folders/_7/4zts0j8d4413ps8jq10scqsr0000gn/T/ipykernel_91179/620480848.py:1: DtypeWarning: Columns (13,14,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Resources/allzipcodes_collision_weather_df.csv")


In [81]:
df.columns

Index(['row_id', 'crash_datetime', 'crash_day_of_week', 'crash_hour_category',
       'street_address', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'vehicle_type_code_1',
       'vehicle_type_code_2', 'borough', 'zip_code', 'latitude', 'longitude',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'Population Density',
       'Population', 'contributing_factor', 'number_of_deaths',
       'number_of_injuries', 'death_rate', 'injuries_rate',
       'severity_of_accident', 'month', 'year', 'day_of_week', 'hour', 'date',
       'tempmax', 'tempmin', 'temp', 'humidity', 'precip', 'preciptype',
       'snow', 'windspeed', 'visibility', 'severerisk', 'sunrise', 'sunset',
       'icon', 'temperature_category', 'precip_category', 'humid_category',
       'wind_level'],
   

In [13]:
df.set_index("row_id",inplace=True)

In [27]:
df['Population'].dtype

dtype('float64')

Goal: Predicting Severity of Accident

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer

# Select relevant features and target variable
features_cat = ['crash_day_of_week', 'crash_hour_category','vehicle_type_code_1',
       'vehicle_type_code_2', 'contributing_factor','icon']
features_num = ['temp', 'windspeed','precip','Population']

y = df['severity_of_accident']
y_dense = LabelBinarizer().fit_transform(y)


# Encode categorical features
label_encoder = LabelEncoder()
for feature in features_cat:
    df[feature] = label_encoder.fit_transform(df[feature])

# Impute missing values
imputer = SimpleImputer(strategy='mean')
df[features_cat] = imputer.fit_transform(df[features_cat])

# Make a column transformer object that scales (using StandardScaler) the non-categorical columns
# and one hot encodes (using OneHotEncoder) the three categorical columns
preprocess = make_column_transformer(
    (StandardScaler(),features_num, ),
    (OneHotEncoder(categories="auto",drop="first"),features_cat, )
)

#Generate the independent variable df
X = preprocess.fit_transform(df).toarray()
X = imputer.fit_transform(X)

#Get x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = train_test_split(X, y_dense, test_size=0.2, random_state=42)

In [64]:
#check the shape
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(83025, 95) (83025, 6)
(20757, 95) (20757, 6)


Create a result df that stores the model score result

In [ ]:
import pandas as pd
import numpy as np
results_df = pd.DataFrame(np.zeros(shape=(6,8)))
results_df.index=[1,2,3,4,5,6]
results_df.columns = ["description","training accuracy","testing accuracy","precision","recall","f1_score","AUC","AP"]
results_df.index.rename("Model",inplace=True)
results_df.description = ["Model1","Model2","Model3","Model4","Model5","Model6"]
results_df

ValueError: Length mismatch: Expected axis has 6 elements, new values have 3 elements

Build Model 1 - OneVsRestClassifier

In [77]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

model1 = OneVsRestClassifier(SVC(probability = True, random_state=42, max_iter = 500))
model1.fit(x_train,y_train)

print(model1.score(x_train,y_train))
print(model1.score(x_test,y_test))


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScale

0.16268593797049083
0.16669075492604904


In [79]:
prediction_probabilities_test = model1.predict_proba(x_test)
prediction_probabilities_test

KeyboardInterrupt: 

Model 1 Metrics

In [72]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.metrics import average_precision_score,roc_auc_score

y_test_pred = model1.predict(x_test)
accuracy_training = model1.score(x_train,y_train)
accuracy_testing = model1.score(x_test,y_test)
recall = recall_score(y_test,y_test_pred, average = "weighted")
precision = precision_score(y_test,y_test_pred, average = "weighted") 
f1 = f1_score(y_test,y_test_pred, average = "weighted")

prediction_probabilities_test = model1.predict_proba(x_test)

y_score = prediction_probabilities_test[:, 1]
y_score_reshaped = np.reshape(y_score, (-1, 1))
ap = average_precision_score(y_test, y_score_reshaped)
# auc = roc_auc_score(y_test, y_score_reshaped, multi_class = 'ovr')



print("Training accuracy: ",accuracy_training)
print("Testing  accuracy: ",accuracy_testing)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1-Score: ",f1)
# print("AUC: ",auc)
print("Average Precision: ",ap)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AttributeError: This 'OneVsRestClassifier' has no attribute 'predict_proba'

Model 2 - SDGClassifier + grid search

In [43]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
#Set up the hyperparameter options in param_grid
param_grid = {
    'class_weight': ["balanced"], 
    'random_state':[42],
    'max_iter': [1000, 2000],
    'loss': ["log_loss"],
    "penalty": ["l1","l2"],
}

#Do the search
gs_clf = GridSearchCV(SGDClassifier(),param_grid, n_jobs = -1, verbose = 3)
gs_clf.fit(x_train, np.ravel(y_train))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 2/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l2, random_state=42;, score=0.443 total time=   9.7s
[CV 3/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l2, random_state=42;, score=0.483 total time=   9.9s
[CV 1/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l2, random_state=42;, score=0.501 total time=  10.3s
[CV 1/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l1, random_state=42;, score=0.595 total time=  16.2s
[CV 2/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l1, random_state=42;, score=0.578 total time=  16.9s
[CV 4/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l2, random_state=42;, score=0.552 total time=   9.5s
[CV 5/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l2, random_state=42;, score=0.520 total time=   9.9s
[CV 4/5] END class_weight=balanced, loss=log_loss, 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV 5/5] END class_weight=balanced, loss=log_loss, max_iter=1000, penalty=l1, random_state=42;, score=0.615 total time= 1.1min


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV 5/5] END class_weight=balanced, loss=log_loss, max_iter=2000, penalty=l1, random_state=42;, score=0.616 total time= 1.8min
CPU times: user 11.9 s, sys: 553 ms, total: 12.5 s
Wall time: 2min 26s


GridSearchCV(estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'], 'loss': ['log_loss'],
                         'max_iter': [1000, 2000], 'penalty': ['l1', 'l2'],
                         'random_state': [42]},
             verbose=3)

In [44]:
gs_clf.best_score_, gs_clf.best_params_ 

(0.6007828967178559,
 {'class_weight': 'balanced',
  'loss': 'log_loss',
  'max_iter': 2000,
  'penalty': 'l1',
  'random_state': 42})

In [46]:
model_2 = SGDClassifier(class_weight = "balanced", penalty = 'l1',random_state=42, max_iter=2000,loss="log_loss")
model_2.fit(x_train, np.ravel(y_train))

SGDClassifier(class_weight='balanced', loss='log_loss', max_iter=2000,
              penalty='l1', random_state=42)

In [48]:
y_test_pred = model_2.predict(x_test)
cfm = confusion_matrix(y_test,y_test_pred)

accuracy_training = model_2.score(x_train,y_train)
accuracy_testing = model_2.score(x_test,y_test)
recall = recall_score(y_test,y_test_pred, average = "weighted")
precision = precision_score(y_test,y_test_pred, average = "weighted") 
f1 = f1_score(y_test,y_test_pred, average = "weighted")

prediction_probabilities_test = model_2.predict_proba(x_test)

y_score = prediction_probabilities_test[:, 1]
y_score_reshaped = np.reshape(y_score, (-1, 1))


ap = average_precision_score(y_test, y_score_reshaped)



print("Confusion Matrix: \n",cfm)
print("Training accuracy: ",accuracy_training)
print("Testing  accuracy: ",accuracy_testing)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1-Score: ",f1)
print("Average Precision: ",ap)

Confusion Matrix: 
 [[    8    23     5     6     1     4]
 [  246 10699   747   770   286   104]
 [    7    40    10     6     3     0]
 [  234  4856   568  1385   228    73]
 [   27   298    54    43    22     2]
 [    0     0     0     0     1     1]]
Training accuracy:  0.5860403492923818
Testing  accuracy:  0.584140290022643
Precision:  0.6388752999016409
Recall:  0.584140290022643
F1-Score:  0.5641888399935125
Average Precision:  0.001644107425436052


Model 3 - Randomforest

In [51]:
%%time
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score,make_scorer
parameters = {
     'n_estimators':[800], #the number of trees
     'min_samples_split': [100, 200],
    'class_weight': ["balanced"],
     'min_samples_leaf': [10,20] #
}
gs_clf = GridSearchCV(RandomForestClassifier(random_state=42),parameters,cv=5,n_jobs=-1,
                      scoring='f1')
gs_clf.fit(x_train, np.ravel(y_train))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._sco

CPU times: user 1min 20s, sys: 1.72 s, total: 1min 21s
Wall time: 7min 51s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': ['balanced'],
                         'min_samples_leaf': [10, 20],
                         'min_samples_split': [100, 200],
                         'n_estimators': [800]},
             scoring='f1')

In [52]:
gs_clf.best_score_, gs_clf.best_params_ 

(nan,
 {'class_weight': 'balanced',
  'min_samples_leaf': 10,
  'min_samples_split': 100,
  'n_estimators': 800})

In [53]:
from sklearn.ensemble import RandomForestClassifier
model_3 = RandomForestClassifier(random_state=42,n_estimators=30,max_depth=6,min_samples_leaf= 10,min_samples_split= 100,class_weight="balanced")
model_3.fit(x_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=6,
                       min_samples_leaf=10, min_samples_split=100,
                       n_estimators=30, random_state=42)

In [54]:
y_test_pred = model_3.predict(x_test)
cfm = confusion_matrix(y_test,y_test_pred)

accuracy_training = model_3.score(x_train,y_train)
accuracy_testing = model_3.score(x_test,y_test)
recall = recall_score(y_test,y_test_pred, average = "weighted")
precision = precision_score(y_test,y_test_pred, average = "weighted") 
f1 = f1_score(y_test,y_test_pred, average = "weighted")

prediction_probabilities_test = model_3.predict_proba(x_test)

y_score = prediction_probabilities_test[:, 1]
y_score_reshaped = np.reshape(y_score, (-1, 1))


ap = average_precision_score(y_test, y_score_reshaped)



print("Confusion Matrix: \n",cfm)
print("Training accuracy: ",accuracy_training)
print("Testing  accuracy: ",accuracy_testing)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1-Score: ",f1)
print("Average Precision: ",ap)

Confusion Matrix: 
 [[  14    9    8   10    3    3]
 [2013 4137 1688 3001 1758  255]
 [  15    8   13   15   12    3]
 [1099 1372 1035 2718  989  131]
 [  67   64   85  116  105    9]
 [   1    0    0    0    0    1]]
Training accuracy:  0.3439205058717254
Testing  accuracy:  0.3366575131281014
Precision:  0.6231420594596287
Recall:  0.3366575131281014
F1-Score:  0.4248591461332949
Average Precision:  0.0016593502716512431


Model 4 - Gradient Boosting Classifier

In [56]:
from sklearn.ensemble import GradientBoostingClassifier

sample_weight = np.array([8 if i == 1 else 1 for i in y_train])


model_4 = GradientBoostingClassifier(min_samples_split=100,
                                     max_depth=8,
                                 min_samples_leaf=100,
                                 n_estimators=400,
                                 subsample=0.6)


model_4.fit(x_train,y_train,sample_weight=sample_weight)



GradientBoostingClassifier(max_depth=8, min_samples_leaf=100,
                           min_samples_split=100, n_estimators=400,
                           subsample=0.6)

In [57]:
y_test_pred = model_4.predict(x_test)
cfm = confusion_matrix(y_test,y_test_pred)

accuracy_training = model_4.score(x_train,y_train)
accuracy_testing = model_4.score(x_test,y_test)
recall = recall_score(y_test,y_test_pred, average = "weighted")
precision = precision_score(y_test,y_test_pred, average = "weighted") 
f1 = f1_score(y_test,y_test_pred, average = "weighted")

prediction_probabilities_test = model_4.predict_proba(x_test)

y_score = prediction_probabilities_test[:, 1]
y_score_reshaped = np.reshape(y_score, (-1, 1))

ap = average_precision_score(y_test, y_score_reshaped)


print("Confusion Matrix: \n",cfm)
print("Training accuracy: ",accuracy_training)
print("Testing  accuracy: ",accuracy_testing)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1-Score: ",f1)
print("Average Precision: ",ap)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix: 
 [[    0    32     0    15     0     0]
 [    4 11129     8  1702     9     0]
 [    0    50     0    16     0     0]
 [    1  5264    11  2061     7     0]
 [    1   349     0    96     0     0]
 [    0     1     0     1     0     0]]
Training accuracy:  0.714062029509184
Testing  accuracy:  0.635448282507106
Precision:  0.5969568390806567
Recall:  0.635448282507106
F1-Score:  0.5941871390164997
Average Precision:  0.001959461883663663


Model 5 - Neural Network 

In [61]:
from sklearn.neural_network import MLPClassifier
model5 = MLPClassifier(
    solver='adam',
    activation='relu',
    alpha=1e-5,
    hidden_layer_sizes=80,
    max_iter = 500,
    learning_rate= 'constant',
    random_state= 42)

model5.fit(x_train,y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=80, max_iter=500, random_state=42)

In [62]:
test_probs = model5.predict_proba(x_test)
test_pred = model5.predict(x_test)

from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
from sklearn.metrics import roc_auc_score,average_precision_score
from sklearn.metrics import accuracy_score

accuracy_training=model5.score(x_train,y_train)
accuracy_testing=model5.score(x_test,y_test)
print("Training accuracy: ",accuracy_training)
print("Testing  accuracy: ",accuracy_testing)
precision = precision_score(y_test,test_pred,average='weighted')
recall = recall_score(y_test,test_pred,average='weighted')
#auc = roc_auc_score(y_test,test_pred)
f1 = f1_score(y_test,test_pred,average='weighted')
ap = average_precision_score(y_test,test_probs)

print("precision: ",precision)
print("recall: ",recall)
print("f1 score: ",f1)
#print("auc",auc)
print("ap",ap)

Training accuracy:  0.6677265883769948
Testing  accuracy:  0.6413739943151708


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision:  0.6027175554531926
recall:  0.6413739943151708
f1 score:  0.5931496986869265
ap 0.21111024073622842
